# The Problem: What are by age what are the preferred listening times of users (Morning/Afternoon/Night)

In [ ]:
import numpy

Rule of Thumb: 2/3 Rule

Age → Time (12 neurons)

- Complex patterns: Age-time preferences have subtle, non-linear relationships
- Single input: Age needs more processing to find patterns
- Reasoning: Young vs old listening habits require enough neurons to model curves